In [5]:
import tensorflow.compat.v1 as tf
from tensorflow import feature_column
from tensorflow.keras import layers
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

In [6]:
seed = 0
# a = tf.Variable(tf.random_normal([1], dtype=tf.float64, seed=0))
# b = tf.Variable(tf.random_normal([1], dtype=tf.float64, seed=0))

# tensorflow, numpy 랜덤 값을 설정합니다.
tf.set_random_seed(seed)
np.random.seed(seed)

In [7]:
#doginfo.csv파일 데이터를 pandas를 이용해 읽어옵니다.
dog_data=pd.read_csv("doginfo.csv")

dog_data = dog_data.dropna(axis=0)
print(dog_data)

       kindNum neuterYn sexCd  weight  noticeDays  age2  processState
0        128.0        N     F    7.46          10    12             0
1        114.0        N     M    7.00          14     1             1
2        114.0        U     M    4.50          11     2             0
3         67.0        N     M   10.00           8     1             0
4        114.0        N     M    6.00           8     4             0
...        ...      ...   ...     ...         ...   ...           ...
22787    114.0        N     M    1.00          10     0             1
22788    114.0        N     M    1.00          10     0             1
22789    114.0        N     M    1.00          10     0             1
22790    128.0        U     F    6.00          12     3             0
22791    114.0        N     M    3.50          10     0             0

[22777 rows x 7 columns]


In [8]:
def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis = 1)
        data = data.drop(column, axis = 1)
    return data

In [22]:
dummy_columns = ["kindNum", "neuterYn", "sexCd"]
data = dummy_data(dog_data, dummy_columns)
data.head()

data = np.array(data, dtype = np.float64)
print(data)

[[ 7.46 10.   12.   ...  1.    0.    0.  ]
 [ 7.   14.    1.   ...  0.    1.    0.  ]
 [ 4.5  11.    2.   ...  0.    1.    0.  ]
 ...
 [ 1.   10.    0.   ...  0.    1.    0.  ]
 [ 6.   12.    3.   ...  1.    0.    0.  ]
 [ 3.5  10.    0.   ...  0.    1.    0.  ]]


In [27]:
a = data[:, :3]
b = data[:, 4:]

# print(a)
# print(b)

#numpy 배열에서 데이터 변화요인(kindCd, neuterYn, sexCd, weight, noticeDays, age2)으로 사용할 데이터를 뽑아냅니다.
xData = np.concatenate([a, b], axis = 1)

print(xData)
type(xData)
xData.shape

[[ 7.46 10.   12.   ...  1.    0.    0.  ]
 [ 7.   14.    1.   ...  0.    1.    0.  ]
 [ 4.5  11.    2.   ...  0.    1.    0.  ]
 ...
 [ 1.   10.    0.   ...  0.    1.    0.  ]
 [ 6.   12.    3.   ...  1.    0.    0.  ]
 [ 3.5  10.    0.   ...  0.    1.    0.  ]]


(22777, 112)

In [28]:
#numpy배열에서 결과(입양여부)로 사용할 데이터를 뽑아냅니다.
yData=data[:,[3]]
# yData = yData.astype('int32')
print(yData)
type(yData)
yData.dtype

[[0.]
 [1.]
 [0.]
 ...
 [1.]
 [0.]
 [0.]]


dtype('float64')

In [29]:
# #뽑아낸 데이터를 tensorflow로 처리하기 위해 placeholder를 만듭니다.
X=tf.placeholder(tf.float64, [None, 112])
Y=tf.placeholder(tf.float64, [None, 1])
# #다변인 선형 회귀 모델의 기울기와 y절편을 임의의 값으로 초기화합니다.
a=tf.Variable(tf.random_normal([112, 1], mean = 0.1, stddev = 0.01, dtype=tf.float64))#6행 1열 난수를 발생시킵니다.
b=tf.Variable(tf.random_normal([1], dtype=tf.float64)) #1행 1열 난수를 발생시킵니다.
print(X)
print(Y)
# print(a)

Tensor("Placeholder:0", shape=(?, 112), dtype=float64)
Tensor("Placeholder_1:0", shape=(?, 1), dtype=float64)


In [55]:
# #행렬의 곱셈 연산을 이용해 다변인 성형 회귀 모델의 가설 식을 세웁니다. ->예측값
# y=tf.matmul(X, a)+b
# #y=tf.clip_by_value(y, 1e-5, 1-(1e-5))

# #오차 함수를 만듭니다.
# loss= tf.reduce_mean(tf.square(y-Y))
# #loss= tf.clip_by_value(loss, 1e-5, 1-(1e-5))

# #경사하강법 알고리즘을 사용해서 오차 함수 결과를 최소로 하는 식을 만듭니다. 
# gradient_descent=tf.train.GradientDescentOptimizer(0.1).minimize(loss)


# # 시그모이드 방정식을 만듭니다.
y = tf.sigmoid(tf.matmul(X, a) + b)
# # 오차 함수
# loss = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1 - y))
# 경사 하강 알고리즘
# # gradient_descent = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

# gradient_descent = tf.train.GradientDescentOptimizer(learning_rate=1e-2).minimize(loss)

# y=tf.nn.softmax(tf.matmul(X, a)+ b)
# cost= tf.reduce_mean(-tf.reduce_sum(Y*tf.log(y), reduction_indices=1))
cost = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1 - y))
optimizer=tf.train.AdamOptimizer(0.0005).minimize(cost)

predicted = tf.cast(y >= 0.5, dtype = tf.float64)

# 예상값이 실제값과 일치하는 정도(정확도)를 계산합니다.
# tf.equal(): tensorflow에서 인수로 지정된 값이 같으면 True, 다르면 False 결과를 tf.float64로 캐스팅 시키고 시행된 전체 평균을 계산합니다.
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float64))  # 정확도 (0 또는 1) 

with tf.Session() as sess:  # 세션 구간을 만듦
    sess.run(tf.global_variables_initializer())
    for i in range(10001):
        # count : 학습 횟수, a1 : 기울기(공부 시간), asess.run(cost, feed_dict={X:xData, Y:yData}))
        a_, b_, cost_, _ = sess.run([a, b, cost, optimizer], feed_dict = {X:xData, Y:yData})
        if i % 2000 == 0:
            print(sess.run(y, feed_dict={X:xData}))
             print('count:{}, b = {}, loss = {}'.format(i, b_, cost_)) # 학습시킨 데이터 출력
#     saver=tf.train.Saver()
#     save_path=saver.save(sess, './dogsaved.ckpt')
#     print('학습된 모델을 저장했습니다.')
    new_x = np.array([114, 0, 1, 6.3, 10, 1], dtype=np.float64).reshape(1, 6)  # reshape() 함수를 이용해서 1행 2열인 2차원 numpy 배열을 만듭니다.
    print(new_x)
    # sigmoid() 함수 연산 결과에 테스트 데이터를 대입시켜 연산합니다.
    result, new_y = sess.run([predicted, y], feed_dict = {X:new_x})
#     print('품종 : %d, 중성화 : %d' % (new_x[:, 0], new_x[:, 1]), end=',')
    print('합격 여부(sigmoid() 함수 실행 결과를 0.5 이상은 1, 미만은 0으로 반환) : %d, 합격 확률(sigmoid() 함수 실행 결과) : %f)' % (result, new_y))
    
#     for i in range(11):
#         new_x = np.array([0, i]).reshape(1, 6)  # 과외 시간만 계속 늘림
    result, new_y = sess.run([predicted, y], feed_dict={X:new_x})
#         print('품종 : %d, 과외 시간 : %2d, ' % (new_x[:, 0], new_x[:, 1]), end=',')
    print('입양 여부 : %s, 입양 확률 : %8.4f%%' % ('합격' if result == 1 else '불합격', new_y*100))
#     print(sess.run(accuracy, feed_dict = {Y : yData}))

# #init=tf.initialize_all_variables()
# sess=tf.Session()
# #sess.run(init)
# sess.run(tf.global_variables_initializer())
# for step in range(2001):
# #     print(sess.run(y, feed_dict={X : xData}))
#     sess.run(optimizer, feed_dict={X:xData, Y:yData})
# #     sess.run(optimizer)
#     if step % 200 == 0:
#         print(step, sess.run(cost, feed_dict={X:xData, Y:yData}), sess.run(a), sess.run(b))
# #     if step % 200 == 0:
# #         print(step, sess.run(a), sess.run(b), {}, a3 = {}, a4 = {}, a5 = {}, a6 = {}, b = {}, loss = {}'.format(i, a_[0], a_[1], a_[2], a_[3], a_[4], a_[5], b_, cost_)) # 학습시킨 데이터 출력
            

[[0.99999959]
 [0.99999623]
 [0.99999465]
 ...
 [0.99998847]
 [0.99999908]
 [0.99999094]]
count:0, a1 = [0.09335948], a2 = [0.10276074], a3 = [0.09414586], a4 = [0.09662561], a5 = [0.10716552], a6 = [0.10962383], b = [-0.44099247], loss = 6.546480927183287
[[0.43295007]
 [0.49444299]
 [0.4873189 ]
 ...
 [0.45914451]
 [0.47361163]
 [0.44086341]]
count:2000, a1 = [-0.00393971], a2 = [0.10951366], a3 = [-0.04682681], a4 = [-0.02954862], a5 = [0.0758202], a6 = [0.01556908], b = [-0.44331357], loss = 0.6794177657703744
[[0.35893503]
 [0.53972312]
 [0.5049089 ]
 ...
 [0.47377872]
 [0.45800578]
 [0.45176682]]
count:4000, a1 = [-0.00892668], a2 = [0.12818274], a3 = [-0.12961369], a4 = [-0.03542124], a5 = [0.11900924], a6 = [0.00070004], b = [-0.24201116], loss = 0.6752203143225964
[[0.34336569]
 [0.52600868]
 [0.50258035]
 ...
 [0.4817764 ]
 [0.43900718]
 [0.4586505 ]]
count:6000, a1 = [-0.01049463], a2 = [0.11881801], a3 = [-0.15047274], a4 = [-0.03713996], a5 = [0.10076718], a6 = [-0.0031735

In [224]:
# #학습을 시킨 후 학습된 모델을 저장합니다.
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     for i in range(10001):
#         loss_, y_, _ = sess.run([loss, y, gradient_descent], feed_dict={X:xData, Y:yData})
#         if i % 500 ==0:
#             print("count:%6d, loss=%12.3f. print=%6.1f" %(i, loss_, y_[0]))


# with tf.Session() as sess:  # 세션 구간을 만듦
#     sess.run(tf.global_variables_initializer())
#     for i in range(3001):
#         # count : 학습 횟수, a1 : 기울기(공부 시간), a2 : 기울기(과외 시간), b : y절편, loss : 오차 함수
#         a_, b_, loss_, _ = sess.run([a, b, loss, gradient_descent], feed_dict = {X:xData, Y:yData})
#         if i % 300 == 0:
#             print('count : {}, a1 = {}, a2 = {}, a3 = {}, a4 = {}, a5 = {}, a6 = {}, b = {}, loss = {}'.format(i, a_[0], a_[1], a_[2], a_[3], a_[4], a_[5], b_, loss_)) # 학습시킨 데이터 출력
            
# 학습 완료 -----------------------------------------------------------------------------------------------------
#테스트 데이터

#     new_x = np.array([6, 5]).reshape(1, 6)  # reshape() 함수를 이용해서 1행 2열인 2차원 numpy 배열을 만듭니다.
    
#     # sigmoid() 함수 연산 결과에 테스트 데이터를 대입시켜 연산합니다.
#     result, new_y = sess.run([predicted, y], feed_dict = {X:new_x})
#     print('공부 시간 : %d, 과외 시간 : %d' % (new_x[:, 0], new_x[:, 1]))
#     print('합격 여부(sigmoid() 함수 실행 결과를 0.5 이상은 1, 미만은 0으로 반환) : %d, 합격 확률(sigmoid() 함수 실행 결과) : %f)' % (result, new_y))
    
#     for i in range(11):
# #         new_x = np.array([0, i]).reshape(1, 6)  # 과외 시간만 계속 늘림
#         result, new_y = sess.run([predicted, y], feed_dict={X:new_x})
# #         print('품종 : %d, 과외 시간 : %2d, ' % (new_x[:, 0], new_x[:, 1]), end=',')
#         print('입양 여부 : %s, 입양 확률 : %8.4f%%' % ('합격' if result == 1 else '불합격', new_y*100))


# 학습이 완료되면 학습된 모델을 디스크에 저장합니다.
# tf.train.Saver(): 학습된 모델을 디스트로 저장 또는 불로오는 객체를 생성하는 함수
    saver= tf.train.Saver()
#     saver_path=saver.save(sess, './dogsave.ckpt')

IndentationError: unexpected indent (<ipython-input-224-75a16fcc1350>, line 21)